# 作业1-PM2.5预测

## 项目描述

* 本次作业的资料是从行政院环境环保署空气品质监测网所下载的观测资料。
* 希望大家能在本作业实现 linear regression 预测出 PM2.5 的数值。

## 数据集介绍

* 本次作业使用丰原站的观测记录，分成 train set（训练集） 跟 test set（测试集），train set 是丰原站每个月的前 20 天所有资料。test set 则是从丰原站剩下的资料中取样出来。
* train.csv（训练数据）: 每个月前 20 天的完整资料。
* test.csv （测试数据）: 从剩下的资料当中取样出连续的 10 小时为一笔，前九小时的所有观测数据当作 feature（特征），第十小时的 PM2.5 当作 answer。一共取出 240 笔不重複的 test data，请根据 feature 预测这 240 笔的 PM2.5。
* Data 含有 18 项观测数据 AMB_TEMP, CH4, CO, NHMC, NO, NO2, NOx, O3, PM10, PM2.5, RAINFALL, RH, SO2, THC, WD_HR, WIND_DIREC, WIND_SPEED, WS_HR。  


## 项目要求
- 请手动实现 linear regression（线性回归），方法限使用 gradient descent（梯度下降）。
- 禁止使用 numpy.linalg.lstsq


## 数据准备
无

## 环境配置/安装


In [1]:
import pandas as pd
import numpy as np
import math

# 读取数据指定第一行为
data = pd.read_csv("./train.csv",encoding = "utf-8",header=0)
# 读取数据中的 PM2.5 的数据
data_pm25 = data[data["名字"]=="PM2.5"]

#将 获取的数据 生成csv文件保存本地
#data_pm25.to_csv("tdata_PM2.5.csv")

print(data_pm25)
print(data.shape)
print(data_pm25.shape)

              时间  代     名字   0   1   2   3   4   5   6  ...  14  15  16  17  \
9       2014/1/1  伦  PM2.5  26  39  36  35  31  28  25  ...  36  45  42  49   
27      2014/1/2  伦  PM2.5  21  23  30  30  22  18  13  ...  53  43  43  45   
45      2014/1/3  伦  PM2.5  19  25  27  20  16  14  15  ...  32  36  34  45   
63      2014/1/4  伦  PM2.5  27  27  14  20  22  24  26  ...  62  55  56  67   
81      2014/1/5  伦  PM2.5  80  80  76  81  75  66  70  ...  64  73  57  57   
...          ...  ..    ...  ..  ..  ..  ..  ..  ..  ..  ...  ..  ..  ..  ..   
4239  2014/12/16  伦  PM2.5   5   9  17  26  33  32  24  ...   8  16  28  37   
4257  2014/12/17  伦  PM2.5  21  26  26  31  29  21  13  ...  12   6   4  11   
4275  2014/12/18  伦  PM2.5   0   2   2   3   3   3  11  ...  31  31  36  31   
4293  2014/12/19  伦  PM2.5  14  16  13  14  21  19  23  ...  21  21  19  19   
4311  2014/12/20  伦  PM2.5   6   1   1   1   0   0   0  ...   1   7  24  37   

      18  19  20  21  22  23  
9     45

In [94]:
import pandas as pd
import numpy as np
import math
data = pd.read_csv("./train.csv",encoding = "utf-8",header=0)  # data是一个dataframe类型的对象

#print(type(data))

# 去掉前三列数据
data = data.iloc[:,3:]
data[data == 'NR'] = 0
raw_data = data.to_numpy()
month_data = {}
for month in range(12):
    sample = np.empty([18,480])
    for day in range(20):
        sample[:,day*24:(day+1)*24] = raw_data[18*(20*month + day):18*(20 * month +day + 1),:]
    month_data[month] = sample
    
x = np.emath([12*471,18*9],dtype = float)
y = np.emath([12*471,1],dtype = float)

for month in range(12):
    for day in range(20):
        for hour in range(24):
            if day  == 19 and hour > 14:
                continue 
            x[month * 471 + day * 24 + hour,:] = month_data[month][:,day * 24 + hour : day * 24 + hour + 9].reshape(1,-1)
            y[month * 471 + day *24 + hour,0] = month_data[month][9,day*24 + hour +9]
# 标准化
mear_x = np.mean(x,axis=0)
std_x = np.std(x,axis=0)
for i in range(len(x)):
    for j in range(len(x[0])):
        if std_x[j] !=0:
            x[i][j] = (x[i][j] - mean_x[j] / std_x[j])


# 训练集测试集的划分
x_train_set = x[:math.float(len(x) * 0.8),:]
y_tarin_set = x[:math.float(len(x) * 0.8),:]

x_validation = x[:math.float(len(x) * 0.8):,:]
y_validation = x[:math.float(len(x) * 0.8):,:]

#维度
dim = 18 * 9 + 1

w = np.zeros(dim,1)
x_train_set = np.concatenate((np.ones([len(x_train_set),1]),x_train_set),axis=1).astype(float)

learning_rate = 10
iter_time = 30000

adagrad = np.zeros([dim,1])
eps = 0.0001
for t in range(iter_time):
    print(x.shape) #[m,n] * [n,k] = [m,k]
    loss = np.sqrt(sum(np.power(np.dot(x_train_set,w) - y_tarin_set,2))/len(x_train_set))
    if(t % 100 == 0):
        print("迭代次数：%i 损失值：%f"%(t,loss))
        adagrad = (np.dot(x_train_set,np.dot(x_train_set,w) - y_tarin_set))/(loss * len(x_train_set))
        adagrad += (adagrad **2)
        w = w -learning_rate * adagrad/np.sqrt(adagrad + eps) # 设置平滑的方式
np.save('weights.npy',w)


#测试


TypeError: 'module' object is not callable

In [36]:
import pandas as pd
import numpy as np
import math
data = pd.read_csv("./train.csv",encoding = "utf-8",header=0)  # data是一个dataframe类型的对象 读取数据源
data = data.iloc[:,3:]   #保留读取第三列之后的数据
data[data == 'NR'] = 0   #对数据进行处理，将值为NR的转成0
raw_data = data.to_numpy() #将dataframe类型转换成一个numpy类型数组
month_data = {} #定义一个字典类型变量month_data
for month in range(12):
    sample = np.empty([18,480])  #默认返回18行480列的一个array，元素是数值接近于0的随机数
    for day in range(20):
        for

[[1.19420023e-311 1.19419786e-311 0.00000000e+000 ... 1.19420866e-311
  1.19420866e-311 1.19420866e-311]
 [1.19420866e-311 1.19420866e-311 1.19420866e-311 ... 1.19420866e-311
  1.19420866e-311 1.19420866e-311]
 [1.19420866e-311 1.19420866e-311 1.19420866e-311 ... 1.19420866e-311
  1.19420866e-311 1.19420866e-311]
 ...
 [0.00000000e+000 0.00000000e+000 0.00000000e+000 ... 0.00000000e+000
  0.00000000e+000 0.00000000e+000]
 [0.00000000e+000 0.00000000e+000 0.00000000e+000 ... 0.00000000e+000
  0.00000000e+000 0.00000000e+000]
 [4.94065646e-324 0.00000000e+000 0.00000000e+000 ... 0.00000000e+000
  0.00000000e+000 4.94065646e-324]]
[[1.19416081e-311 1.19419786e-311 0.00000000e+000 ... 0.00000000e+000
  2.12199579e-314 0.00000000e+000]
 [1.39067116e-309 0.00000000e+000 0.00000000e+000 ... 1.39067116e-309
  0.00000000e+000 0.00000000e+000]
 [4.94065646e-324 0.00000000e+000 3.23790862e-319 ... 4.94065646e-324
  0.00000000e+000 3.23790862e-319]
 ...
 [7.74860419e-304 7.74860419e-304 7.74860419